In [37]:
import numpy as np
import random


def calc_prob(X, K, pMu, pSigma):
    N = X.shape[0]
    D = X.shape[1]
    Px = np.zeros((N, K))
    for i in range(K):
        Xshift = X-np.tile(pMu[i], (N, 1))
        lambda_flag = np.e**(-5)
        conv = pSigma[i]+lambda_flag*np.eye(D)
        inv_pSigma = np.linalg.inv(conv)
        tmp = np.sum(np.dot(Xshift, inv_pSigma)*Xshift, axis=1)
        coef = (2*np.pi)**(-D/2)*np.sqrt(np.linalg.det(inv_pSigma))
        Px[:, i] = coef*np.e**(-1/2*tmp)
    return Px


def gmm(X, K):       #用array来处理
    threshold = np.e**(-15)
    N = X.shape[0]
    D = X.shape[1]
    rndp = random.sample(np.arange(N).tolist(),K)
    centroids = X[rndp,:]
    pMu = centroids
    pPi = np.zeros((1, K))
    pSigma = np.zeros((K, D, D))
    dist = np.tile(np.sum(X*X, axis=1).reshape(N,1), (1, K))+np.tile(np.sum(pMu*pMu, axis=1), (N, 1))-2*np.dot(X, pMu.T)
    labels = np.argmin(dist,axis=1)
    for i in range(K):
        index = labels == i
        Xk = X[index,:]
        pPi[:,i] = (Xk.shape[0])/N
        pSigma[i] = np.cov(Xk.T)
    Loss = -float("inf")
    while True:
        Px = calc_prob(X, K, pMu, pSigma)
        pGamma = Px*np.tile(pPi, (N, 1))
        pGamma = pGamma/np.tile(np.sum(pGamma, axis=1).reshape(N,1), (1, K))
        Nk = np.sum(pGamma, axis=0)
        pMu = np.dot(np.dot(np.diag(1/Nk), pGamma.T), X)
        pPi = Nk/N
        for i in range(K):
            Xshift = X-np.tile(pMu[i], (N, 1))
            pSigma[i] = np.dot(Xshift.T, np.dot(np.diag(pGamma[:, i]), Xshift))/Nk[i]
        L = np.sum(np.log(np.dot(Px, pPi.T)), axis=0)
        if L-Loss < threshold:
            break
        Loss = L
    return Px,pMu,pSigma,pPi
        

if __name__ == "__main__":        
    Data_list = []
    with open("data.txt", 'r') as file:
        for line in file.readlines():  
            point = []  
            point.append(float(line.split()[0]))  
            point.append(float(line.split()[1]))  
            Data_list.append(point)  
    Data = np.array(Data_list)
    Px,pMu,pSigma,pPi = gmm(Data, 2)
    print(Px,pMu,pSigma,pPi)



[[  3.64439063e-12   1.76862084e-02]
 [  2.45641357e-10   1.73570005e-02]
 [  2.77096255e-18   4.08347636e-02]
 [  5.09051493e-35   7.85590674e-04]
 [  1.80942419e-06   2.22079865e-03]
 [  2.24816473e-19   1.77476569e-02]
 [  5.08582076e-11   2.22120577e-02]
 [  1.04822931e-18   4.83365166e-02]
 [  3.01254234e-27   1.08432096e-02]
 [  1.31538195e-13   5.99109699e-02]
 [  3.89807646e-14   5.75359078e-02]
 [  2.22750844e-09   8.19625603e-03]
 [  2.12747034e-12   3.12630966e-02]
 [  1.03983808e-13   1.66435581e-03]
 [  6.18219822e-14   6.06215205e-02]
 [  8.10760725e-18   3.77733104e-02]
 [  9.40361077e-13   1.82200393e-02]
 [  2.17498889e-07   8.69188765e-03]
 [  4.50117419e-18   2.46238961e-02]
 [  1.20950848e-19   1.47058075e-02]
 [  2.76787479e-21   2.61538691e-02]
 [  1.68815184e-08   1.74197045e-02]
 [  2.41705557e-11   3.07217753e-02]
 [  3.03714124e-10   3.14395994e-02]
 [  7.24913868e-32   2.04731055e-03]
 [  4.16982272e-18   5.23486032e-02]
 [  2.59716883e-14   2.61646982e-02]
 